# README
In this notebook we will go through the workflow of uploading and downloading information from GBEX.
Its mostly uses a default django-rest-framework setup, so anything from the django-rest-framework documentation is probably valid here aswell.
Login is set to "token" based, but other authentication backends can also be configured. See django and/or django-rest-framework documentation.

This tutorial assumes the default database model setup. If you have changed the database model, please make changes to this tutorial as needed.

### Basic API stuff:
Communication happens via the requests package which sends standard http requests:
1. a GET request "gets" information
  1. On a GET you can filter objects by specifying a "params" dictionary (examples below)
2. a POST request creates new objects
  1. In a POST you specify a data dictionary with the fields needed for the object (these can be found in the database schema, read futher down)
3. a PUT or PATCH request updates existing objects (PUT replaces all fields with a new value, PATCH only changes specified fields)
  1. PUT and PATCH requires the same as the POST.


### Database schema can be reviewed at either
1. localhost/api
  1. django rest framework interface, allows login, interaction, but doesnt show data types/info
2. localhost/swagger
  1. swagger interface, does not allow login (you must be logged in from previously visiting GBEX, the admin interface or the django rest framework API), but allows interaction and shows data types
3. localhost/redoc
  1. redoc interface, does not allow login (you must be logged in from previously visiting GBEX, the admin interface or the django rest framework API), but allows interaction and shows data types. I find this interface particularly good for getting a list of fields for each model

In [11]:
#This tutorial requires the following python packages:  
# requests and pytz

!pip install -U requests pytz

Requirement already up-to-date: requests in /home/laeb/.pyenv/versions/3.8.2/envs/gbex_api/lib/python3.8/site-packages (2.23.0)
  Using cached pytz-2020.1-py2.py3-none-any.whl (510 kB)


In [2]:
# The default GBEX setup uses token based authentication.
# To create a token, visit: localhost/admin
# Click "tokens", then "add token"
# Then either enter user id or search for a user (your superuser probably has id 1)
# Then click save and you should then see a "key". Copy and paste that here

auth_token = '25cb2aeb008101c504da971b20ca0215626dcf46'
base_url = 'http://localhost'

In [56]:
# create a session and add the auth token to it
import requests
c = requests.session()
c.headers.update({'Authorization': f'Token {auth_token}'})

In [7]:
# ok step 1, lets create a plasmid
# to do that we need to create a dict describing a plasmid
# visit http://localhost/redoc/#operation/Plasmid_create to see what that dict should look like
# under the Plasmid_create header you can see that you need to supply a dict with the following info:
#    "name": "string, default PL1",
#    "Description": "string, nullable",
#    "responsible": "integer, nullable"
#
# That means that you can give a name, but if you don't, it will be given "PL1", and you can also give a Description and a responsible but those are "nullable", aka not required
# get your userid (e.g. from the admin interface)
my_user_id = 1

data = {
    "Description": "this is a really nice plasmid",
    "responsible": my_user_id,  
}

In [9]:
# create a plasmid. Post requests MUST with a slash otherwise you get error 500
res = c.post(f"{base_url}/api/Plasmid/", data=data)

# print the status code (201 = good)
print(res.status_code)

# show the json response (if succesful, it returns the newly created object)
res.json()

201


{'id': 3,
 'name': 'PL3',
 'Genbank_file': None,
 'created': '2020-05-08T14:10:36.873887+02:00',
 'edited': '2020-05-08T14:10:36.873922+02:00',
 'Description': 'this is a really nice plasmid',
 'responsible': 1}

In [12]:
# now lets create a Strain and link it with the plasmid
# first lets get the plasmid id. In this tutorial we could easily get the Plasmid id from the "res.json()" object, but lets query the API for plasmids created by user 1
plasmid_id = c.get(f"{base_url}/api/Plasmid", params={'responsible': my_user_id}).json()[0]['id']
plasmid_id

1

In [29]:
# then we create a Strain
# redoc tells us we need a dict like this:
#    "Subtype": "string",
#    "Description": "string",
#    "responsible": integer (linking to a user)
#    "Species": integer (linking to a species option)
#    "ParentStrain": integer (linking to a strain)
#    "Plasmids": array of integers
#
# Species is required...ok then we need to create species option
# A SpeciesOption just requires a name
res = c.post(f"{base_url}/api/SpeciesOption/", data={'name': 'ecoli'})
# now the name has to be unique, so if it already exists you will get a returned status_code of 500
# to see more details on the error, try running this cell twice
if res.status_code > 300:
    print(res.content.decode()[:200])  # just showing the first 200 character of the error message because its loooong
else:
    print(res.json())

IntegrityError at /api/SpeciesOption/
duplicate key value violates unique constraint "GBEX_app_speciesoption_name_key"
DETAIL:  Key (name)=(ecoli) already exists.


Request Method: POST
Request URL: h


In [32]:
# now get the id of the option
options = c.get(f"{base_url}/api/SpeciesOption/", params={'name': 'ecoli'}).json()
ecoli_option = options[0]['id']

In [33]:
# ok back to creating the strain
data = {
    "responsible": my_user_id,
    "Species": ecoli_option,
    "Plasmids": [plasmid_id]  # there could be more than 1 plasmids here
}
res = c.post(f"{base_url}/api/Strain/", data=data)
print(res.status_code)
res.json()

201


{'id': 5,
 'name': 'ST5',
 'Genbank_file': None,
 'created': '2020-05-08T15:05:02.375540+02:00',
 'edited': '2020-05-08T15:05:02.375574+02:00',
 'Subtype': None,
 'Description': None,
 'responsible': 1,
 'Species': 2,
 'ParentStrain': None,
 'Plasmids': [1]}

In [38]:
# ok lets try and upload a genbank file for that strain. First lets take the id from the res.json object
strain_id = res.json()['id']
# and then lets import the upload function from upload.py which is a script in this directory
from upload import upload_a_file_to_gbex
# now the only file I konw you have is this ipynb file, but feel free to change it to something more relevant
upload_a_file_to_gbex("api_example.ipynb", auth_token, f"{base_url}/api/Strain/{strain_id}", "Genbank_file")

Attempting to pre-upload file in 1 chunk(s)
Uploading final chunk
	<Response [201]>: File successfully uploaded and named: 1198824_api_example.ipynb
Attempting to attach file to object
<Response [200]>


{'id': 5,
 'name': 'ST5',
 'Genbank_file': 'http://localhost/downloads/Strain/ST5/1198824_api_example.ipynb',
 'created': '2020-05-08T15:05:02.375540+02:00',
 'edited': '2020-05-08T15:10:34.742940+02:00',
 'Subtype': None,
 'Description': None,
 'responsible': 1,
 'Species': 2,
 'ParentStrain': None,
 'Plasmids': [1]}

In [44]:
# if that succeded you should see you got a 201 and a 200 respsonse and finally you should see the Strain object now with a Genbank_file
# Now what if we wanted to download a file via api:
file_url = c.get(f"{base_url}/api/Strain/", params={'id': strain_id}).json()[0]['Genbank_file']
print(file_url)
actual_file = c.get(file_url)

http://localhost/downloads/Strain/ST5/1198824_api_example.ipynb


In [50]:
# now we actually have the file in "actual_file". But how do we use it?
#we could write to disk by executing the next 2 lines
#with open('save_name.txt', 'wb') as f:
#    f.write(actual_file.content)

# however if you want to just use it directly in python that often requires a file like object
from io import StringIO  # using stringIo to convert the "file" variable into a file-like objext
file_like_object = StringIO(actual_file.text)
file_like_object.readlines()[:5] # show first 5 lines of file

['{\n',
 ' "cells": [\n',
 '  {\n',
 '   "cell_type": "markdown",\n',
 '   "metadata": {},\n']

In [57]:
# now finally I want to show you how to use the datetime required in the Fermentation model
import pytz
from datetime import datetime

In [58]:
time_string = datetime.now().replace(tzinfo=pytz.timezone("Europe/Copenhagen")).replace(microsecond=0).isoformat()

In [59]:
data = {
    "RunDate": time_string,
}
res = c.post(f"{base_url}/api/Fermentation/", data=data)
print(res.status_code)
res.json()

201


{'id': 3,
 'name': 'FE3',
 'Data_file': None,
 'created': '2020-05-08T15:26:46.449234+02:00',
 'edited': '2020-05-08T15:26:46.449257+02:00',
 'RunDate': '2020-05-08T16:36:32+02:00',
 'responsible': None,
 'Strain': None}

In [60]:
# There you go. Now you know EVERYTHING!